In [1]:
#
# NOTE: This machine learning program is for predicting TC formation, using
#       input dataset in the NETCDF format. The program treats different
#       2D input fields as different channels of an image. This specific
#       program requires a set of 12 2D-variables (12-channel image) and
#       consists of three stages
#       - Stage 1: reading NETCDF input and generating (X,y) data with a
#                  given image sizes, which are then saved by pickle;
#       - Stage 2: import the saved pickle (X,y) pair and build a CNN model
#                  with a given training/validation ratio, and then save
#                  the train model under tcg_CNN.model.
#       - Stage 3: import the trained model from Stage 2, and make a list
#                  of prediction from normalized test data.
#
# INPUT: This Stage 3 script reads in the CNN-trained model "tcg_CNN.model"
#        that is generated from Step 2.
#
#        Remarks: Note that the input data for this script must be on the
#        same as in Step 1 with standard 19 vertical
#        levels 1000, 975, 950, 925, 900, 850, 800, 750, 700, 650, 600,
#        550, 500, 450, 400, 350, 300, 250, 200. Also, all field vars must
#        be resized to cover an area of 30x30 around the TC center for the
#        positive data cases.
#        Similar to Step 2, Step 3 needs to also have a large mem
#        allocation so that it can be run properly.
#
# OUTPUT: A list of probability forecasts with the same dimension as the
#        number of input 12-channel images.
#
# HIST: - 01, Nov 22: Created by CK
#       - 02, Nov 22: Modified to optimize it
#       - 05, Jun 23: Rechecked and added F1 score function for a list of model
#       - 12, Jun 23: Customized for ResNet from the CNN functional model
#       - 18, Nov 23: re-designed for better check
#
# AUTH: Chanh Kieu (Indiana University, Bloomington. Email: ckieu@iu.edu)
#
#==========================================================================
import cv2
import tensorflow as tf
import os
from tqdm import tqdm
import netCDF4
import numpy as np
import libtcg_netcdfloader as tcg_loader
import libtcg_utils as tcg_utils
import pickle
import sys

In [2]:
#
# Function to return input data as a numpy array
#
def prepare(filepath):
    IMG_SIZE = 30
    number_channels = 12
    f = netCDF4.Dataset(filepath)
    abv = f.variables['absvprs']
    nx = np.size(abv[0,0,:])
    ny = np.size(abv[0,:,0])
    nz = np.size(abv[:,0,0])
    #print('Dimension of input NETCDF is: ',nx,ny,nz)
    a2 = np.zeros((nx,ny,number_channels))
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,0] = abv[1,j,i]    # abs vort at 950 mb
    rel = f.variables['rhprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,1] = rel[7,j,i]    # RH at 750 mb
    sfc = f.variables['pressfc']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,2] = sfc[j,i]      # surface pressure
    tmp = f.variables['tmpprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,3] = tmp[15,j,i]   # temperature at 400 mb
    tsf = f.variables['tmpsfc']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,4] = tsf[j,i]      # surface temperature
    ugr = f.variables['ugrdprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,5] = ugr[3,j,i]    # u-wind at 900 mb
            a2[i,j,6] = ugr[17,j,i]   # u-wind at 300 mb
    vgr = f.variables['vgrdprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,7] = vgr[3,j,i]    # v-wind at 900 mb
            a2[i,j,8] = vgr[17,j,i]   # v-wind at 300 mb
    hgt = f.variables['hgtprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,9] = hgt[3,j,i]    # geopotential at 850 mb
    wgr = f.variables['vvelprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,10] = wgr[3,j,i]   # w-wind at 900 mb
            a2[i,j,11] = wgr[17,j,i]  # w-wind at 300 mb
    new_array = cv2.resize(a2, (IMG_SIZE, IMG_SIZE))
    #
    # normalize the data
    #
    #print('Number of channels to normalize is: ',number_channels)
    for var in range(number_channels):
        maxvalue = new_array[:,:,var].flat[np.abs(new_array[:,:,var]).argmax()]
        #print('Normalization factor for channel',var,', is: ',abs(maxvalue))
        new_array[:,:,var] = new_array[:,:,var]/abs(maxvalue)
    out_array = np.reshape(new_array, (-1, IMG_SIZE, IMG_SIZE, number_channels))
    #print('reshape new_array returns: ',out_array.shape)
    #input('Enter to continue...')
    return out_array

In [11]:
#
# Visualize the output of the training model (work for jupyter notebook only)
#
def view_history(histories):
    import matplotlib.pyplot as plt
    val_accuracy1 = histories[0]['val_binary_accuracy']
    accuracy1 = histories[0]['binary_accuracy']
    val_accuracy2 = histories[1]['val_binary_accuracy']
    accuracy2 = histories[1]['binary_accuracy']
    val_accuracy3 = histories[2]['val_binary_accuracy']
    accuracy3 = histories[2]['binary_accuracy']
    epochs = np.arange(len(val_accuracy1))
    plt.plot(epochs,val_accuracy1,'r',label="val binary_accuracy resnet20")
    plt.plot(epochs,accuracy1,'r--',label="train binary_accuracy resnet20")
    plt.plot(epochs,val_accuracy2,'b',label="val binary_accuracy resnet22")
    plt.plot(epochs,accuracy2,'b--',label="train binary_accuracy resnet22")
    plt.plot(epochs,val_accuracy3,'g',label="val binary_accuracy resnet40")
    plt.plot(epochs,accuracy3,'g--',label="train binary_accuracy resnet40")
    plt.legend()

    plt.figure()
    val_loss1 = histories[0]['val_loss']
    loss1 = histories[0]['loss']
    val_loss2 = histories[1]['val_loss']
    loss2 = histories[1]['loss']
    val_loss3 = histories[2]['val_loss']
    loss3 = histories[2]['loss']
    plt.plot(epochs,val_loss1,'r',label="val loss resnet20")
    plt.plot(epochs,loss1,'r--',label="train loss resnet20")
    plt.plot(epochs,val_loss2,'b',label="val loss resnet22")
    plt.plot(epochs,loss2,'b--',label="train loss resnet22")
    plt.plot(epochs,val_loss3,'g',label="val loss resnet40")
    plt.plot(epochs,loss3,'g--',label="train loss resnet40")
    plt.legend()
    plt.show()
#
# loop thru all best-saved CNN-trained models and make a prediction. Note that prediction is applied one by one instead 
# of a batch input. 
#
def main(DATADIR="",bestmodels=[],test_sample=10):
    #
    # Test data with an input structure of pos/neg dirs
    #
    CATEGORIES = ["neg", "pos"]
    F1_performance = []
    for bestmodel in bestmodels:
        model = tf.keras.models.load_model(bestmodel)
        prediction_total = 0
        prediction_yes = 0
        prediction_history = []
        truth_history = []
        for category in CATEGORIES:
            path = os.path.join(DATADIR,category)
            for img in tqdm(os.listdir(path)):    
                try:
                    img_dir = DATADIR + '/' + category + '/' + img
                    print('Processing image:', img_dir)
                    #print('Input image dimension is: ',prepare(img_dir).shape)
                    indata, _ = tcg_loader.prepare12channels(img_dir,IMG_SIZE=32)
                    prediction = model.predict([indata])
                    print("TC formation prediction is",prediction,round(prediction[0][0]),CATEGORIES[round(prediction[0][0])])
                    prediction_history.append(prediction[0][0])
                    if round(prediction[0][0]) == 1:
                        prediction_yes += 1
                    if category == "pos":
                        truth_history.append(1)
                    else:
                        truth_history.append(0)
                    prediction_total += 1                        
                    if prediction_total > test_sample:
                        prediction_total = 0
                        break
                except Exception as e:
                    pass   
        #
        # Compute F1 score for each best model and save it
        #
        #print(prediction_history)
        F1_performance.append([bestmodel,tcg_utils.F1_score(truth_history,prediction_history,1,0.5)]) 
    return F1_performance

In [13]:
#
# main fucntion
#
if __name__ == '__main__':
    n = len(sys.argv)
    print("Total arguments input are:", n)
    print("Name of Python script:", sys.argv[0])
    leadtime = "00"
    datadir = "/N/project/hurricane-deep-learning/data/ncep_extracted_binary_18x18/EP/"+leadtime+"/"
    #if n < 3:
    #   print("Need a forecast lead time and datapath to process...Stop")
    #   print("+ Example: tcg_ResNet_p3.py 00 /N/project/hurricane-deep-learning/data/ncep_extracted_binary_30x30/EP/00/")
    #   print("Using the default values: ",leadtime,datadir)
    #leadtime = str(sys.argv[1])
    #datadir = str(sys.argv[2])
    print("Forecast lead time to run is: ",leadtime)
    print("Datapath is: ",datadir)
    #sys.exit()

    history_check = "no"
    if history_check == "yes":
        hist = open("tcg_histories_resnet.pickle","rb")
        histories = pickle.load(hist)
        view_history(histories)

    bestmodels = ["tcg_ResNet20.model_00","tcg_ResNet22.model_00","tcg_ResNet40.model_00"]
    performance = main(DATADIR=datadir,bestmodels=bestmodels,test_sample=100)
    print("Summary of the CNN model performance for forecast lead time: ",leadtime)
    for i in range(len(bestmodels)):
        print("Model:",bestmodels[i]," --- F1, Recall, Presision:",np.round(performance[i][1],2))   

Total arguments input are: 3
Name of Python script: /N/u/ckieu/Carbonate/.local/lib/python3.10/site-packages/ipykernel_launcher.py
Forecast lead time to run is:  00
Datapath is:  /N/project/hurricane-deep-learning/data/ncep_extracted_binary_18x18/EP/00/


  0%|                                                                                                          | 0/447 [00:00<?, ?it/s]

Processing image: /N/project/hurricane-deep-learning/data/ncep_extracted_binary_18x18/EP/00//neg/20150707_12_00_13.4_258.1.nc
1/1 [==============================] - 0s 413ms/step


  0%|▏                                                                                                 | 1/447 [00:01<08:44,  1.18s/it]

TC formation prediction is [[0.24709918]] 0 neg
Processing image: /N/project/hurricane-deep-learning/data/ncep_extracted_binary_18x18/EP/00//neg/20180929_12_00_12.0_293.7.nc
1/1 [==============================] - 0s 27ms/step


...

TC formation prediction is [[0.01276453]] 0 neg
Processing image: /N/project/hurricane-deep-learning/data/ncep_extracted_binary_18x18/EP/00//pos/20070714_12_00_11.2_234.2_2007196N11234.nc
1/1 [==============================] - 0s 26ms/step


  5%|████▎                                                                                          | 100/2205 [01:14<25:44,  1.36it/s]

TC formation prediction is [[0.00588578]] 0 neg
Processing image: /N/project/hurricane-deep-learning/data/ncep_extracted_binary_18x18/EP/00//pos/20061006_12_00_14.2_242.8_2006282N14243.nc
1/1 [==============================] - 0s 26ms/step


  5%|████▎                                                                                          | 100/2205 [01:15<26:31,  1.32it/s]

TC formation prediction is [[0.00452951]] 0 neg
[0.005456021, 0.0152417915, 0.0075387917, 0.007153584, 0.005063329, 0.0068790494, 0.003453274, 0.004224569, 0.01266368, 0.004027651, 0.021302756, 0.005239463, 0.0076514515, 0.009219069, 0.014529801, 0.009949304, 0.0068052527, 0.0067373305, 0.014389928, 0.0055703265, 0.005461719, 0.0033359143, 0.004504927, 0.006549427, 0.0047007296, 0.01027357, 0.0071173506, 0.00478695, 0.0067105917, 0.012804816, 0.006995592, 0.0069647944, 0.005233407, 0.0052917274, 0.0061504617, 0.0035496426, 0.004793681, 0.012033654, 0.010317941, 0.0031509523, 0.008180425, 0.007836209, 0.011857316, 0.006021321, 0.012621253, 0.0061016874, 0.006691969, 0.006450944, 0.011455376, 0.009150229, 0.0057114447, 0.010186562, 0.0067596105, 0.012251271, 0.013878083, 0.0044685407, 0.010810354, 0.009207682, 0.0055739386, 0.0067362958, 0.0077870213, 0.005570311, 0.008735613, 0.009676685, 0.0055578654, 0.004744405, 0.009927767, 0.009109947, 0.0073799733, 0.011243988, 0.0046209195, 0.008